In [2]:
%pip install pulp

  Using cached PuLP-2.9.0-py3-none-any.whl.metadata (5.4 kB)
Using cached PuLP-2.9.0-py3-none-any.whl (17.7 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pulp

In [19]:
#Declaring constants
NUM_COMPONENTS = 11

In [46]:
# dict for the parameters [height, width] for each component
param_dict = {
    # For square-shaped components, "Component name": [component_length, component_width] 
    # For circular-shaped, "Component name": component_radius
    'chipset':[5,5], 
    'cpu':[3,3], 
    'coolfan':5, 
    'cmos':3, 
    'bios':[3,3], 
    'powercon':[2,1], 
    'usb':[2,1], 
    'ram':[10,2]
}

# define the LP
problem = pulp.LpProblem('motherboard', pulp.LpMinimize)

# define variables
p1 = 2
p2 = 1
x = pulp.LpVariable.dicts('x',range(1, NUM_COMPONENTS+1), lowBound=0, cat='Continuous')
y = pulp.LpVariable.dicts('y',range(1, NUM_COMPONENTS+1), lowBound=0, cat='Continuous')
W = pulp.LpVariable('W', lowBound=0, cat='Continuous')
H = pulp.LpVariable('H', lowBound=0, cat='Continuous')

# objective function
problem += 2*W + 2*H

# Adding horizontal constraints
problem += x[1] - p1 >= 0
problem += x[1] + p1 + param_dict['chipset'][1] + param_dict['coolfan'] <= x[3]
problem += x[4] - p1 >= 0
problem += x[4] + p1 + 2*param_dict['cmos'] <= x[5]
problem += x[5] + p1 + param_dict['bios'][1] <= x[6]
problem += x[6] + p1 + param_dict['powercon'][1] <= x[7]
problem += x[7] + p1 + param_dict['usb'][1] <= x[8]
problem += x[3] + p1 + param_dict['coolfan'] <= x[8]
problem += x[8] + p2 + param_dict['ram'][1] <= x[9]
problem += x[9] + p2 + param_dict['ram'][1] <= x[10]
problem += x[10] + p2 + param_dict['ram'][1] <= x[11]
problem += x[11] + p2 + param_dict['ram'][1] <= W

# Adding vertical constraints
problem += y[4] + p1 + param_dict['cmos'] >= 0
problem += y[4] + p1 + param_dict['cmos'] <= y[1]
problem += y[1] + p1 + param_dict['chipset'][0] <= H
problem += y[5] - p1 >= 0
problem += y[5] + p1 + param_dict['bios'][0] <= y[1]
problem += y[6] - p1 >= 0
problem += y[6] + p1 + param_dict['powercon'][0] + param_dict['coolfan'] <= y[3]
problem += y[3] + p1 + param_dict['coolfan'] <= H
problem += y[7] - p1 >= 0
problem += y[7] + p1 + param_dict['usb'][0] + param_dict['coolfan'] <= y[3]
problem += y[8] - p1 >= 0
problem += y[8] + param_dict['ram'][0] <= H
problem += y[9] - p1 >= 0
problem += y[9] + param_dict['ram'][0] <= H
problem += y[10] - p1 >= 0
problem += y[10] + param_dict['ram'][0] <= H
problem += y[11] - p1 >= 0
problem += y[11] + param_dict['ram'][0] <= H

#alignment constraints
x[2] + 0.5*param_dict['cpu'][1] == x[3]
x[3] + param_dict['coolfan'] == x[6]
y[2] + 0.5*param_dict['cpu'][0] == y[3]

# solve
problem.solve(pulp.PULP_CBC_CMD(msg=0))

# create dictionary with the solutions
solution = {}
for var in problem.variables():
    solution[var.name] = var.varValue


print("Optimised Perimeter of motherboard: ", 2*(solution["W"] + solution["H"]), "cm")
print("Width of motherboard:", solution['W'],  "Height of motherboard:", solution["H"])
print(f"Coordinates of Chipset (component 1): ",(solution["x_1"] , solution["y_1"]))
#TODO: what's gg on with the CPU?
# print(f"Coordinates of CPU (component 2): ",(solution["x_2"] , solution["y_2"]))
print(f"Coordinates of Cooling Fan (component 3): ",(solution["x_3"] , solution["y_3"]))
print(f"Coordinates of CMOS (component 4): ",(solution["x_4"] , solution["y_4"]))
print(f"Coordinates of BIOS (component 5): ",(solution["x_5"] , solution["y_5"]))
print(f"Coordinates of Power Converter (component 6): ",(solution["x_6"] , solution["y_6"]))
print(f"Coordinates of USB Converter (component 7): ",(solution["x_7"] , solution["y_7"]))
print(f"Coordinates of RAM Memory Stick 1 (component 8): ",(solution["x_8"] , solution["y_8"]))
print(f"Coordinates of RAM Memory Stick 2 (component 9): ",(solution["x_9"] , solution["y_9"]))
print(f"Coordinates of RAM Memory Stick 3 (component 10): ",(solution["x_10"] , solution["y_10"]))
print(f"Coordinates of RAM Memory Stick 4 (component 11): ",(solution["x_11"] , solution["y_11"]))



Optimised Perimeter of motherboard:  102.0 cm
Width of motherboard: 33.0 Height of motherboard: 18.0
Coordinates of Chipset (component 1):  (2.0, 11.0)
Coordinates of Cooling Fan (component 3):  (14.0, 11.0)
Coordinates of CMOS (component 4):  (2.0, 0.0)
Coordinates of BIOS (component 5):  (10.0, 2.0)
Coordinates of Power Converter (component 6):  (15.0, 2.0)
Coordinates of USB Converter (component 7):  (18.0, 2.0)
Coordinates of RAM Memory Stick 1 (component 8):  (21.0, 2.0)
Coordinates of RAM Memory Stick 2 (component 9):  (24.0, 2.0)
Coordinates of RAM Memory Stick 3 (component 10):  (27.0, 2.0)
Coordinates of RAM Memory Stick 4 (component 11):  (30.0, 2.0)
